# User Inputs

In [1]:
year = 2018

db_name = 'Season_Stats.sqlite3'

# Load Packages

In [2]:
import pandas as pd
import numpy as np
import os
import sqlite3
from data_functions import *
import re

# Update Team Efficiency

In [ ]:
team_off = pd.read_html('https://www.footballoutsiders.com/stats/teamoff')[0].iloc[:, 1:]

colnames_team_off = {
    1: 'team',
    2: 'overall_dvoa',
    3: 'last_year',
    4: 'weighted',
    5: 'rank',
    6: 'pass_off',
    7: 'pass_rank',
    8: 'rush_off',
    9: 'rush_rank',
    10: 'non_adj',
    11: 'non_adj_2',
    12: 'non_adj_3',
    13: 'variance',
    14: 'var_rank',
    15: 'schedule_diff',
    16: 'sched_rank'
}

team_off.columns = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
team_off = team_off.rename(columns=colnames_team_off).dropna().reset_index(drop=True)
team_off['year'] = year
    
    
cols = ['overall_dvoa', 'pass_off', 'rush_off', 'variance' , 'schedule_diff']
for col in cols:
    team_off[col] = team_off[col].apply(name_clean)
    
team_off = team_off[['team', 'year', 'overall_dvoa', 'last_year', 'pass_off', 
                     'rush_off', 'variance', 'schedule_diff']]

team_convert = {
    'ARI': 'ARI',
    'ATL': 'ATL',
    'BAL': 'BAL',
    'BUF': 'BUF',
    'CAR': 'CAR',
    'CHI': 'CHI',
    'CIN': 'CIN',
    'CLE': 'CLE',
    'DAL': 'DAL',
    'DEN': 'DEN',
    'DET': 'DET',
    'GB': 'GNB',
    'HOU': 'HOU',
    'IND': 'IND',
    'JAX': 'JAX',
    'KC': 'KAN',
    'LACH': 'LAC',
    'SD': 'LAC',
    'LAC': 'LAC',
    'LAR': 'LAR', 
    'LARM': 'LAR',
    'STL': 'LAR',
    'MIA': 'MIA',
    'MIN': 'MIN',
    'NE': 'NWE',
    'NO': 'NOR',
    'NYG': 'NYG',
    'NYJ': 'NYJ',
    'OAK': 'OAK',
    'PHI': 'PHI',
    'PIT': 'PIT',
    'SEA': 'SEA',
    'SF': 'SFO',
    'TB': 'TAM',
    'TEN': 'TEN',
    'WAS': 'WAS'
}

team_off['team'] = team_off.team.map(team_convert)

append_to_db(team_off, db_name=db_name, table_name='Team_Efficiency', if_exist='append')

# Update Traditional Team Stats

In [ ]:
#==========
# Team Passing Stats
#==========

url_team = 'http://www.footballdb.com/stats/teamstat.html?lg=NFL&yr=' + str(year) + '&type=reg&cat=P&group=O&conf='
data_team = pd.read_html(url_team)[0]

cols= {
    'Team': 'team', 
    'Att': 'tm_pass_att',
    'Cmp': 'tm_pass_complete', 
    'Pct': 'tm_pass_complete_pct',
    'Yds': 'tm_pass_yds',
    'TD': 'tm_pass_td',
    'Int': 'tm_pass_int',
    'Sack': 'tm_sack',
    'Loss': 'tm_loss_sack_yds',
    'Rate': 'tm_qb_rate',
    'NetYds': 'tm_net_pass_yds',
    'Yds/G': 'tm_pass_yperg',
    'year': 'year'
}

df_team = data_team.rename(columns=cols).drop('Gms', axis=1)

In [ ]:
def unique_list(list_strings):
    ulist = []
    [ulist.append(x) for x in list_strings if x not in ulist]
    return ulist

team_names = []
for team in df_team.team:
    name_separate = re.sub(r"(\w)([A-Z])", r"\1 \2", team)
    team_name = ' '.join(unique_list(name_separate.split()))
    team_names.append(team_name)
    
df_team['team'] = team_names

In [ ]:
team_to_abb = {
    
    'Arizona Cardinals': 'ARI',
    'Atlanta Falcons': 'ATL',
    'Baltimore Ravens': 'BAL',
    'Buffalo Bills': 'BUF',
    'Carolina Panthers': 'CAR',
    'Chicago Bears': 'CHI',
    'Cincinnati Bengals': 'CIN',
    'Cleveland Browns': 'CLE',
    'Dallas Cowboys': 'DAL',
    'Denver Broncos': 'DEN',
    'Detroit Lions': 'DET',
    'Green Bay Packers': 'GNB',
    'Houston Texans': 'HOU',
    'Indianapolis Colts': 'IND',
    'Jacksonville Jaguars': 'JAX',
    'Kansas City Chiefs': 'KAN',
    'Los Angeles Chargers LA': 'LAC',
    'San Diego Chargers': 'LAC',
    'Los Angeles Rams LA': 'LAR',
    'Los Angeles Rams': 'LAR',
    'St. Louis Rams': 'LAR',
    'Miami Dolphins': 'MIA',
    'Minnesota Vikings': 'MIN',
    'New England Patriots': 'NWE',
    'New Orleans Saints': 'NOR',
    'New York Giants NY': 'NYG',
    'New York Jets NY': 'NYJ',
    'Oakland Raiders': 'OAK',
    'Philadelphia Eagles': 'PHI',
    'Pittsburgh Steelers': 'PIT',
    'Seattle Seahawks': 'SEA',
    'San Francisco 49ers': 'SFO',
    'Tampa Bay Buccaneers': 'TAM',
    'Tennessee Titans': 'TEN',
    'Tennessee Oilers': 'TEN',
    'Washington Redskins': 'WAS'
}

df_team.team = df_team.team.map(team_to_abb)
df_team['year'] = year
dfteam = df_team.reset_index(drop=True)

In [ ]:
#==========
# Team Rushing Stats
#==========
url_team = 'https://www.footballdb.com/stats/teamstat.html?lg=NFL&yr=' + str(year) + '&type=reg&cat=R&group=O&conf='
data_team_rush = pd.read_html(url_team)[0]

cols= {
    'Team': 'team', 
    'Att': 'tm_rush_att',
    'Yds': 'tm_rush_yds',
    'Avg': 'team_rush_avg_att',
    'TD': 'tm_rush_td',
    'FD': 'tm_rush_first_downs',
    'Yds/G': 'tm_rush_yperg',
    'year': 'year'
}

df_rush = data_team_rush.rename(columns=cols).drop('Gms', axis=1)

team_names = []
for team in df_rush.team:
    name_separate = re.sub(r"(\w)([A-Z])", r"\1 \2", team)
    team_name = ' '.join(unique_list(name_separate.split()))
    team_names.append(team_name)
    
df_rush['team'] = team_names

df_rush.team = df_rush.team.map(team_to_abb)

df_rush['year'] = year
df_rush = df_rush.reset_index(drop=True)

In [ ]:
#==========
# Overall Team Stats
#==========

url_team = 'https://www.footballdb.com/stats/teamstat.html?lg=NFL&yr=' + str(year) + '&type=reg&cat=W&group=O&conf='
data_team_fd = pd.read_html(url_team)

df_fd = data_team_fd[0].T.reset_index(drop=True).T

cols= {
    0: 'team', 
    1: 'games',
    2: 'tm_rush_fd',
    3: 'tm_pass_fd',
    4: 'tm_pen_fd',
    5: 'tm_tot_fd',
    6: 'tm_third_down_att',
    7: 'tm_third_down_made',
    8: 'tm_third_down_pct',
    9: 'tm_fourth_down_att',
    10: 'tm_fourth_down_made',
    11: 'tm_fourth_down_pct'
}

df_fd = df_fd.rename(columns=cols)
df_fd = df_fd[['team', 'tm_rush_fd', 'tm_pass_fd', 'tm_pen_fd', 'tm_tot_fd', 'tm_third_down_att',
               'tm_third_down_made', 'tm_third_down_pct']]

df_fd['year'] = year

team_names = []
for team in df_fd.team:
    name_separate = re.sub(r"(\w)([A-Z])", r"\1 \2", team)
    team_name = ' '.join(unique_list(name_separate.split()))
    team_names.append(team_name)
    
df_fd['team'] = team_names

df_fd.team = df_fd.team.map(team_to_abb)

In [ ]:
#==========
# Merging All Team Data
#==========

df_team = pd.merge(df_team, df_rush, how='inner', left_on=['team', 'year'], right_on=['team', 'year'])
df_team = pd.merge(df_team, df_fd, how='inner', left_on=['team', 'year'], right_on=['team', 'year'])

df_team['total_tm_td'] = df_team.tm_pass_td + df_team.tm_rush_td
df_team['total_tm_yds'] = df_team.tm_pass_yds + df_team.tm_rush_yds

df_team.iloc[:, 1:] = df_team.iloc[:, 1:].astype('float')

append_to_db(team_off, db_name=db_name, table_name='Team_Offensive_Stats', if_exist='append')

# Update Offensive Line

In [26]:
oline = pd.read_html('https://www.footballoutsiders.com/stats/ol')[0]

oline_df = oline.T.reset_index(drop=True).T

cols = {
    0: 'run_block_rank',
    1: 'team', 
    2: 'adjust_line_yds',
    3: 'rb_yds', 
    4: 'power_success_pct',
    5: 'power_rank',
    6: 'stuffed_pct',
    7: 'stuffed_rank',
    8: 'second_level_yds',
    9: 'second_level_rank',
    10: 'open_field_yds',
    11: 'open_field_rank',
    12: 'pass_block_rank', 
    13: 'sacks_allowed', 
    14: 'adjusted_sack_rate'
}

oline_df = oline_df.rename(columns=cols)

oline_df['year'] = year
        
oline_df['power_success_pct'] = oline_df.power_success_pct.apply(name_clean)
oline_df['stuffed_pct'] = oline_df.stuffed_pct.apply(name_clean)
oline_df['adjusted_sack_rate'] = oline_df.adjusted_sack_rate.apply(name_clean)

oline_df = oline_df[['team', 'year', 'adjust_line_yds', 'rb_yds', 'power_success_pct',
                     'stuffed_pct', 'second_level_yds', 'open_field_yds', 'pass_block_rank',
                     'sacks_allowed', 'adjusted_sack_rate']]

team_convert = {
    
    'ARI': 'ARI',
    'ATL': 'ATL',
    'BAL': 'BAL',
    'BUF': 'BUF',
    'CAR': 'CAR',
    'CHI': 'CHI',
    'CIN': 'CIN',
    'CLE': 'CLE',
    'DAL': 'DAL',
    'DEN': 'DEN',
    'DET': 'DET',
    'GB': 'GNB',
    'HOU': 'HOU',
    'IND': 'IND',
    'JAX': 'JAX',
    'KC': 'KAN',
    'LAC': 'LAC',
    'SD': 'LAC',
    'LAR': 'LAR',
    'STL': 'LAR',
    'MIA': 'MIA',
    'MIN': 'MIN',
    'NE': 'NWE',
    'NO': 'NOR',
    'NYG': 'NYG',
    'NYJ': 'NYJ',
    'OAK': 'OAK',
    'PHI': 'PHI',
    'PIT': 'PIT',
    'SEA': 'SEA',
    'SF': 'SFO',
    'TB': 'TAM',
    'TEN': 'TEN',
    'WAS': 'WAS'
}

oline_df['team'] = oline_df.team.map(team_convert)
for col in oline_df.columns:
    try:
        oline_df[col] = oline_df[col].astype('float')
    except:
        pass

append_to_db(team_off, db_name=db_name, table_name='OLine_Stats', if_exist='append')

# QB for Position Players

In [36]:
#==========
# Scraping Statistical and ADP Data
#==========

# pulling passing statistics
url_player = 'https://www.pro-football-reference.com/years/' + str(year) + '/passing.htm'
data_player = pd.read_html(url_player)[0]

# pulling historical player adp
url_adp = 'http://www03.myfantasyleague.com/' + str(year) + '/adp?COUNT=100&POS=QB&ROOKIES=0&INJURED=1&CUTOFF=1&FRANCHISES=-1&IS_PPR=-1&IS_KEEPER=-1&IS_MOCK=-1&TIME='
data_adp = pd.read_html(url_adp)[1]

In [70]:
#--------
# Cleaning Player Statistical Data
#--------

df_player = data_player.iloc[:, 1:]
df_player['year'] = year

colnames_pass = {
    'Player': 'player',
    'Tm': 'team',
    'Pos': 'position',
    'Age': 'qb_age',
    'G': 'qb_games',
    'GS': 'qb_games_started',
    'QBrec': 'qb_record',
    'Cmp': 'qb_complete',
    'Att': 'qb_att',
    'Cmp%': 'qb_complete_pct',
    'Yds': 'qb_yds',
    'TD': 'qb_tds',
    'TD%': 'qb_td_pct',
    'Int': 'int',
    'Int%': 'int_pct',
    'Lng': 'qb_long',
    'Y/A': 'yd_per_att',
    'AY/A': 'adj_yd_per_att',
    'Y/C': 'yd_per_comp',
    'Y/G': 'qb_yd_per_game',
    'Rate': 'qb_rating',
    'QBR': 'qbr',
    'Sk': 'qb_sacks',
    'Yds.1': 'yds_lost_sack',
    'NY/A': 'net_yd_per_att',
    'ANY/A': 'adj_net_yd_per_att',
    'Sk%': 'sack_pct',
    '4QC': 'fourth_qt_comeback',
    'GWD': 'game_winning_drives'
}

df_player = df_player.rename(columns=colnames_pass)
df_player = df_player[df_player.player != 'Player']

df_player = df_player[['player', 'team', 'year', 'qb_age', 'qb_games_started', 'qb_att', 'qb_rating', 
                       'qbr', 'qb_complete', 'qb_complete_pct', 'qb_yds', 'qb_yd_per_game', 'qb_tds', 
                       'qb_td_pct', 'int', 'int_pct', 'qb_long', 'yd_per_att', 'yd_per_comp',
                       'adj_net_yd_per_att', 'adj_yd_per_att', 'net_yd_per_att', 'qb_sacks', 'sack_pct',
                       'fourth_qt_comeback', 'game_winning_drives']]

# cleaning player name and stat categories
df_player['player'] = df_player.player.apply(name_clean)

# fill missing values with zero
df_player = df_player.fillna(0)

for col in df_player.columns:
    try:
        df_player[col] = df_player[col].astype('float')
    except:
        pass

In [86]:
#==========
# Cleaning Player ADP Data
#==========

df_adp = data_adp.iloc[1:, :]

colnames_adp = {
    0: 'rank', 
    1: 'player',
    2: 'qb_avg_pick',
    3: 'min_pick',
    4: 'max_pick',
    5: 'drafts_selected'
}

df_adp = df_adp.rename(columns=colnames_adp)

df_adp['year'] = year

# selecting team and player name information from combined string
df_adp['team'] = df_adp.player.apply(team_select)
df_adp['player'] = df_adp.player.apply(name_select)
df_adp['player'] = df_adp.player.apply(name_clean)

# format and rename columns
df_adp = df_adp[['player', 'team', 'year', 'qb_avg_pick']].dropna()

# convert year to float and move back one year to match with stats
df_adp['year'] = df_adp.year.astype('float')

In [90]:
#==========
# Merging and Formatting All Player Data
#==========

# merge adp and player data
player_df = pd.merge(df_player, df_adp, how = 'left', left_on = ['player', 'year'], right_on = ['player', 'year'])

# filter players who threw more than 50 passes in season
player_df = player_df[player_df.qb_att > 50].reset_index(drop=True)

# ensure all teams have same abbreviations for matching
adp_to_player_teams = {
    
    'ARI': 'ARI',
    'ATL': 'ATL',
    'BAL': 'BAL',
    'BUF': 'BUF',
    'CAR': 'CAR',
    'CHI': 'CHI',
    'CIN': 'CIN',
    'CLE': 'CLE',
    'DAL': 'DAL',
    'DEN': 'DEN',
    'DET': 'DET',
    'GBP': 'GNB',
    'HOU': 'HOU',
    'IND': 'IND',
    'JAC': 'JAX',
    'KCC': 'KAN',
    'LAC': 'LAC',
    'SDC': 'LAC',
    'LAR': 'LAR',
    'RAM': 'LAR',
    'MIA': 'MIA',
    'MIN': 'MIN',
    'NEP': 'NWE',
    'NOS': 'NOR',
    'NYG': 'NYG',
    'NYJ': 'NYJ',
    'OAK': 'OAK',
    'PHI': 'PHI',
    'PIT': 'PIT',
    'SEA': 'SEA',
    'SFO': 'SFO',
    'TBB': 'TAM',
    'TEN': 'TEN',
    'WAS': 'WAS'
}

player_df['team_y'] = player_df['team_y'].map(adp_to_player_teams)

# update old team names to LA team names
la_update = {
    'STL': 'LAR',
    'SDG': 'LAC'
}

la_teams = player_df[(player_df.team_x == 'SDG') | (player_df.team_x == 'STL')]
la_teams['team_x'] = la_teams.team_x.map(la_update)
player_df.update(la_teams)

# fill in null teams with current team
player_df['team_y'] = player_df.team_y.fillna(player_df.team_x)

# create flag if player switched teams
player_df['qb_new_team'] = player_df['team_x'] != player_df['team_y']
player_df['qb_new_team'] = player_df.qb_new_team.map({True: 1, False: 0})

# keep current team 
player_df = player_df.drop('team_y', axis=1)
player_df = player_df.rename(columns = {'team_x': 'team'})

# fill null adp values
player_df['qb_avg_pick'] = player_df.qb_avg_pick.fillna(200)

# select columns
player_df = player_df[['player', 'team', 'year', 'qb_age', 'qb_games_started', 'qb_att', 'qb_rating', 
                       'qb_complete', 'qb_complete_pct', 'qb_yds', 'qb_yd_per_game', 'qb_tds', 
                       'qb_td_pct', 'int', 'int_pct', 'qb_long', 'yd_per_att', 'yd_per_comp', 'net_yd_per_att', 
                       'adj_net_yd_per_att', 'adj_yd_per_att', 'qb_sacks', 'sack_pct', 'qb_avg_pick']]

player_df.iloc[:, 2:] = player_df.iloc[:, 2:].astype('float')

# Define a lambda function to compute the weighted mean:
weighted_mean = lambda x: np.average(x, weights=player_df.loc[x.index, "qb_att"])

# Define a dictionary with the functions to apply for a given column:
functions = {
    'qb_age': weighted_mean,
    'qb_games_started': weighted_mean,
    'qb_att': ['sum'],
    'qb_rating': weighted_mean,
    'qb_complete': ['sum'],
    'qb_complete_pct': weighted_mean,
    'qb_yds': ['sum'],
    'qb_yd_per_game': weighted_mean,
    'qb_tds': ['sum'],
    'qb_td_pct': weighted_mean,
    'int': ['sum'],
    'int_pct': weighted_mean,
    'qb_long': weighted_mean,
    'yd_per_att': weighted_mean,
    'yd_per_comp': weighted_mean,
    'net_yd_per_att': weighted_mean,
    'adj_net_yd_per_att': weighted_mean,
    'adj_yd_per_att': weighted_mean,
    'net_yd_per_att': weighted_mean,
    'qb_sacks': ['sum'],
    'sack_pct': weighted_mean,
    'qb_avg_pick': weighted_mean
}

# group based on agg functions on the team and year levels for attachment to WR df
grouped_df = player_df.drop(['player'], axis=1)
grouped_df = grouped_df.groupby(['team', 'year'], group_keys=False).agg(functions).reset_index()
grouped_df.columns = grouped_df.columns.droplevel(1)

In [91]:
grouped_df

,team,year,qb_age,qb_games_started,qb_att,qb_rating,qb_complete,qb_complete_pct,qb_yds,qb_yd_per_game,...,int_pct,qb_long,yd_per_att,yd_per_comp,net_yd_per_att,adj_net_yd_per_att,adj_yd_per_att,qb_sacks,sack_pct,qb_avg_pick
0,ARI,2018.0,22.923077,9.461538,416.0,65.569231,237.0,57.007692,2442.0,163.103846,...,4.353846,67.307692,5.888462,10.342308,4.698077,3.447692,4.511538,41.0,8.938462,181.156538
1,ATL,2018.0,33.000000,14.000000,538.0,107.600000,376.0,69.900000,4307.0,307.600000,...,1.100000,75.000000,8.000000,11.500000,6.960000,7.530000,8.600000,40.0,6.900000,58.070000
2,BAL,2018.0,29.662857,8.165714,525.0,84.394667,317.0,60.365714,3487.0,216.668000,...,1.739048,71.834286,6.639048,10.989333,6.034438,5.951657,6.539048,30.0,5.407048,148.506533
3,BUF,2018.0,24.653465,6.383663,404.0,56.876733,218.0,53.972525,2394.0,157.385891,...,4.966337,59.111386,5.955941,11.031931,4.789307,3.067302,4.029455,39.0,8.776733,200.000000
4,CAR,2018.0,29.000000,14.000000,471.0,94.200000,320.0,67.900000,3395.0,242.500000,...,2.800000,82.000000,7.200000,10.600000,6.360000,6.150000,7.000000,29.0,5.800000,47.580000
5,CHI,2018.0,25.336264,10.329670,455.0,93.515385,299.0,65.701758,3329.0,212.535165,...,3.099780,65.991209,7.299780,11.116044,6.522923,6.297868,7.066374,30.0,6.185275,117.994396
6,CIN,2018.0,29.388778,8.851703,499.0,86.941483,306.0,61.309218,3341.0,200.377355,...,2.597194,45.777555,6.677756,10.943487,5.885531,5.727756,6.504609,30.0,5.641683,154.963828
7,CLE,2018.0,24.036585,9.617886,492.0,87.745325,302.0,61.394919,3538.0,231.714024,...,2.648171,62.717480,7.171748,11.713618,6.225264,5.996524,6.902642,38.0,7.095732,127.062622
8,DAL,2018.0,25.000000,14.000000,457.0,94.000000,309.0,67.600000,3337.0,238.400000,...,1.800000,90.000000,7.300000,10.800000,5.940000,5.900000,7.300000,51.0,10.000000,103.270000
9,DEN,2018.0,30.000000,14.000000,501.0,82.100000,311.0,62.100000,3396.0,242.600000,...,2.400000,64.000000,6.800000,10.900000,5.940000,5.490000,6.300000,32.0,6.000000,157.170000
